In [ ]:
df = df[[f'volume_{target_stock}','Bitcoin']]
df_col = df.columns.tolist()

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import copy
from darts import TimeSeries
from darts.models import TFTModel
from darts.utils.likelihood_models import QuantileRegression
from darts.dataprocessing.transformers import Scaler
from darts.metrics import mae, rmse, smape, mape, r2_score, mse, rmsle
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

warnings.filterwarnings("ignore")

# === Step 1: Load and Prepare Data === #
print("\n[INFO] Loading data...")
df = pd.read_csv("../data/Stock_market_data/clean_stock_data_with_time_index.csv", parse_dates=["date"], index_col="date")
df = df.sort_index()
raw_main_df = copy.deepcopy(df)
print(f"[DEBUG] df shape: {df.shape}, index: {df.index.min()} to {df.index.max()}")
print(f"[DEBUG] df NaN: {df.isnull().sum().sum()}")

# === Step 2: Select Target Stock === #
target_stock = "AAPL"
drop_cols = [f"open_{target_stock}", f"low_{target_stock}", f"high_{target_stock}"]
df.drop(columns=drop_cols, inplace=True)
target_df = df[[f"close_{target_stock}"]].rename(columns={f"close_{target_stock}": "target"})
df.drop(columns=f"close_{target_stock}", inplace=True)
df = df[[f'volume_{target_stock}','Bitcoin']]
df_col = df.columns.tolist()

# === Step 3: Time Feature Engineering === #
features = pd.DataFrame(index=target_df.index)
features["month"] = features.index.month
features["day_of_week"] = features.index.dayofweek
features["day_of_month"] = features.index.day
features["week"] = features.index.isocalendar().week.values.astype(int)
features["quarter"] = features.index.quarter

def encode_cyclical(df, col, max_val):
    df[f"{col}_sin"] = np.sin(2 * np.pi * df[col] / max_val)
    df[f"{col}_cos"] = np.cos(2 * np.pi * df[col] / max_val)
    df.drop(columns=[col], inplace=True)

for col, max_v in zip(["month", "day_of_week", "day_of_month", "week", "quarter"], [12, 7, 31, 53, 4]):
    encode_cyclical(features, col, max_v)

features = pd.concat([features, df[df_col]], axis=1)
print(f"[DEBUG] features shape : {features.shape}, type: {type(features)} ")
print(f"[DEBUG] features index: {features.index.min()} to {features.index.max()}")
print(f"[DEBUG] features NaN: {features.isnull().sum().sum()}")

# === Step 4: Interpolate Missing Values === #
full_df = pd.concat([target_df, features], axis=1).resample("D").interpolate(method="linear")
print(f"[DEBUG] full_df shape: {full_df.shape}, index: {full_df.index.min()} to {full_df.index.max()}")
print(f"[DEBUG] full_df NaN after interpolate: {full_df.isnull().sum().sum()}")

# === Step 5: Convert to TimeSeries === #
raw_target = TimeSeries.from_dataframe(full_df, value_cols="target", freq="D")
raw_covariates = TimeSeries.from_dataframe(full_df, value_cols=features.columns.tolist())
print(f"[DEBUG] raw_target len: {len(raw_target)}, freq: {raw_target.freq_str}, NaNs: {np.isnan(raw_target.values()).sum()}")
print(f"[DEBUG] raw_target shape : {raw_target.shape}, type: {type(raw_target)} ")
print(f"[DEBUG] raw_target index-len: {raw_target.time_index.min()} to {raw_target.time_index.max()}")
print(f"[DEBUG] raw_covariates shape : {raw_covariates.shape}, type: {type(raw_covariates)} ")
print(f"[DEBUG] raw_covariates index-len: {raw_covariates.time_index.min()} to {raw_covariates.time_index.max()}")

# === Step 6: Define Parameters for Sliding Window === #
output_len = 15
input_len = output_len * 6
num_windows = 5
window_step = 30

if input_len <= output_len:
    raise ValueError("[ERROR] input_len must be greater than output_len to create a valid window.")

print(f"[INFO] input_len={input_len}, output_len={output_len}, window_step={window_step}, total_len={len(raw_target)}")

all_metrics = []

# === Step 7: Loop Over Sliding Windows === #
for i in range(num_windows):
    print(f"\n[INFO] Window {i+1}/{num_windows}")

    train_end = len(raw_target) - (num_windows - i) * window_step - output_len
    val_start = train_end - input_len
    val_end = train_end + output_len

    print(f"[DEBUG] train_end={train_end}, val_start={val_start}, val_end={val_end}")

    if val_start < 0 or val_end + output_len > len(raw_target):
        print(f"[WARNING] Skipping window {i+1} due to index bounds.")
        continue

    train_target = raw_target[:train_end]
    val_target = raw_target[val_start:val_end]
    train_covariates = raw_covariates[:train_end]
    val_covariates = raw_covariates[val_start:val_end + output_len]

    print(f"[DEBUG] train_target: {train_target.start_time()} to {train_target.end_time()}, len={len(train_target)}")
    print(f"[DEBUG] val_target: {val_target.start_time()} to {val_target.end_time()}, len={len(val_target)}")
    print(f"[DEBUG] val_covariates: {val_covariates.start_time()} to {val_covariates.end_time()}, len={len(val_covariates)}")

    # === Step 8: Scaling per Window === #
    t_scaler = Scaler()
    f_scaler = Scaler()
    train_target_scaled = t_scaler.fit_transform(train_target)
    val_target_scaled = t_scaler.transform(val_target)
    train_cov_scaled = f_scaler.fit_transform(train_covariates)
    val_cov_scaled = f_scaler.transform(val_covariates)

    print(f"[DEBUG] Scaled train_target shape: {train_target_scaled.shape}, val_target shape: {val_target_scaled.shape}")
    print(f"[DEBUG] Scaled train_covariates shape: {train_cov_scaled.shape}, val_covariates shape: {val_cov_scaled.shape}")

    # === Step 9: Model Definition === #
    model = TFTModel(
        input_chunk_length=input_len,
        output_chunk_length=output_len,
        hidden_size=64,
        lstm_layers=2,
        dropout=0.3,
        batch_size=64,
        n_epochs=5,
        num_attention_heads=4,
        likelihood=QuantileRegression([0.1, 0.5, 0.9]),
        pl_trainer_kwargs={"callbacks": [EarlyStopping(monitor="val_loss", patience=3)]},
        force_reset=True
    )

    model.fit(train_target_scaled, future_covariates=train_cov_scaled,
              val_series=val_target_scaled, val_future_covariates=val_cov_scaled, verbose=True)

    # === Step 10: Forecasting === #
    forecast = model.predict(n=output_len, series=val_target_scaled,
                             future_covariates=val_cov_scaled, num_samples=100)
    forecast_p10 = t_scaler.inverse_transform(forecast.quantile_timeseries(0.1))
    forecast_p50 = t_scaler.inverse_transform(forecast.quantile_timeseries(0.5))
    forecast_p90 = t_scaler.inverse_transform(forecast.quantile_timeseries(0.9))

    # === Step 11: Evaluate === #
    actual = val_target.slice_intersect(forecast_p50)
    print(f"[DEBUG] forecast_p50: {forecast_p50.start_time()} to {forecast_p50.end_time()}, len={len(forecast_p50)}")
    print(f"[DEBUG] actual: {actual.start_time() if len(actual) > 0 else 'N/A'} to {actual.end_time() if len(actual) > 0 else 'N/A'}, len={len(actual)}")

    if len(actual) == 0:
        print(f"[WARNING] Skipping metrics for window {i+1}: No overlapping timestamps.")
        continue

    metrics = {
        "Window": i + 1,
        "MAPE": mape(actual, forecast_p50),
        "SMAPE": smape(actual, forecast_p50),
        "MAE": mae(actual, forecast_p50),
        "RMSE": rmse(actual, forecast_p50),
        "R2": r2_score(actual, forecast_p50),
        "MSE": mse(actual, forecast_p50)
    }
    all_metrics.append(metrics)

    # === Step 12: Confidence Interval Plot === #
    plt.figure(figsize=(10, 5))
    plt.fill_between(forecast_p50.time_index, forecast_p10.values().squeeze(), forecast_p90.values().squeeze(), alpha=0.3, label="Confidence Interval")
    forecast_p50.plot(label="Forecast (p50)", color="blue")
    actual.plot(label="Actual", color="black")
    plt.title(f"Forecasting Window {i+1}")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # === Step 13: Loss Curve === #
    if hasattr(model.trainer, 'callback_metrics'):
        loss = model.trainer.callback_metrics
        print(f"[INFO] Final val_loss for window {i+1}: {loss.get('val_loss')}")

# === Step 14: Aggregate Performance === #
metrics_df = pd.DataFrame(all_metrics)
print("\n[INFO] Aggregated Performance:")
print(metrics_df.describe())

# === Step 15: Save Metrics === #
metrics_df.to_csv("aggregated_window_metrics.csv", index=False)
print("[INFO] Saved aggregated metrics to 'aggregated_window_metrics.csv'")


GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

   | Name                              | Type                             | Params
----------------------------------------------------------------------------------------
0  | train_metrics                     | MetricCollection                 | 0     
1  | val_metrics                       | MetricCollection                 | 0     
2  | input_embeddings                  | _MultiEmbedding                  | 0     
3  | static_covariates_vsn             | _VariableSelectionNetwork        | 0     
4  | encoder_vsn                       | _VariableSelectionNetwork        | 24.0 K
5  | decoder_vsn                       | _VariableSelectionNetwork        | 22.0 K
6  | static_context_grn                | _GatedResidualNetwork            | 16.8 K
7  | static_context_hidden_encoder_grn | _GatedResidualNetwork            | 16.8 K
8  | static


[INFO] Loading data...
[DEBUG] df shape: (2847, 181), index: 2014-01-02 00:00:00 to 2025-04-28 00:00:00
[DEBUG] df NaN: 0
[DEBUG] features shape : (2847, 12), type: <class 'pandas.core.frame.DataFrame'> 
[DEBUG] features index: 2014-01-02 00:00:00 to 2025-04-28 00:00:00
[DEBUG] features NaN: 0
[DEBUG] full_df shape: (4135, 13), index: 2014-01-02 00:00:00 to 2025-04-28 00:00:00
[DEBUG] full_df NaN after interpolate: 0
[DEBUG] raw_target len: 4135, freq: D, NaNs: 0
[DEBUG] raw_target shape : (4135, 1, 1), type: <class 'darts.timeseries.TimeSeries'> 
[DEBUG] raw_target index-len: 2014-01-02 00:00:00 to 2025-04-28 00:00:00
[DEBUG] raw_covariates shape : (4135, 12, 1), type: <class 'darts.timeseries.TimeSeries'> 
[DEBUG] raw_covariates index-len: 2014-01-02 00:00:00 to 2025-04-28 00:00:00
[INFO] input_len=90, output_len=15, window_step=30, total_len=4135

[INFO] Window 1/5
[DEBUG] train_end=3970, val_start=3880, val_end=3985
[DEBUG] train_target: 2014-01-02 00:00:00 to 2024-11-14 00:00:00,

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|██████████| 62/62 [00:20<00:00,  3.01it/s, loss=0.0253, train_loss=0.0277, val_loss=0.038]

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


...

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|██████████| 64/64 [00:21<00:00,  3.02it/s, loss=0.0294, train_loss=0.0257, val_loss=0.0457]


GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  3.02it/s]
[DEBUG] forecast_p50: 2025-03-30 00:00:00 to 2025-04-13 00:00:00, len=15
[DEBUG] actual: N/A to N/A, len=0
[WARNING] Skipping metrics for window 5: No overlapping timestamps.

[INFO] Aggregated Performance:


ValueError: Cannot describe a DataFrame without columns